<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250414%5D%20Federated%20Learning%20Simulation%20(FedProx)%20on%20WSN-DS%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FedAvg-DL // with clean MLP-LSTM model 5 classes on WSN-DS**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Flatten
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.utils import to_categorical

In [ ]:
# Load dataset
dataset = pd.read_csv("dataset_WSN-DS.csv") #WSN-DS 5 classes dataset

# Preprocessing: Handle missing values (if any)
dataset = dataset.dropna() # Remove missing values

In [ ]:
dataset

,id,Time,Is_CH,who CH,Dist_To_CH,ADV_S,ADV_R,JOIN_S,JOIN_R,SCH_S,SCH_R,Rank,DATA_S,DATA_R,Data_Sent_To_BS,dist_CH_To_BS,send_code,Expaned Energy,Class
0,101000,50,1,101000,0.00000,1,0,0,25,1,0,0,0,1200,48,130.08535,0,2.46940,0
1,101001,50,0,101044,75.32345,0,4,1,0,0,1,2,38,0,0,0.00000,4,0.06957,0
2,101002,50,0,101010,46.95453,0,4,1,0,0,1,19,41,0,0,0.00000,3,0.06898,0
3,101003,50,0,101044,64.85231,0,4,1,0,0,1,16,38,0,0,0.00000,4,0.06673,0
4,101004,50,0,101010,4.83341,0,4,1,0,0,1,25,41,0,0,0.00000,3,0.06534,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374656,201096,1003,0,201051,6.98337,0,5,1,0,0,1,7,96,0,67,170.14779,3,0.15974,0
374657,201097,1003,0,201037,29.32867,0,5,1,0,0,1,31,39,0,24,82.21043,2,0.06877,0
374658,201098,1003,0,201095,18.51963,0,5,1,0,0,1,17,55,0,31,139.26438,1,0.09437,0
374659,201099,1003,0,201051,8.55001,0,5,1,0,0,1,3,96,0,65,158.27492,3,0.16047,0


**Data Preprocessing**

In [ ]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

In [ ]:
print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [340066  14596  10049   6638   3312]


In [ ]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
!pip install -U imbalanced-learn

**Data Balancing utilized SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE
# Balance dataset using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [340066 340066 340066 340066 340066]


**Split data into 80% Training and 20% Testing**

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

**The hybrid MLP + LSTM model**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, TimeDistributed, InputLayer
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

# Convert target to categorical (one-hot encoding)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Reshape the input data for LSTM compatibility
# Assuming the input features need to be reshaped into a 3D array: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the hybrid MLP + LSTM model with regularization
def build_hybrid_model(input_shape, num_classes):
    model = Sequential()

    # Input Layer
    model.add(InputLayer(input_shape=input_shape))

    # LSTM Layer
    model.add(LSTM(64, activation='tanh', return_sequences=False, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization

    # MLP Layers
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.3))  # Dropout for regularization

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile the model
input_shape = (X_train.shape[1], X_train.shape[2])  # Adjusted for LSTM input format
num_classes = y_train.shape[1]
model = build_hybrid_model(input_shape, num_classes)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 263s 6ms/step - accuracy: 0.8736 - loss: 0.3694 - val_accuracy: 0.9056 - val_loss: 0.2301
Epoch 2/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 265s 6ms/step - accuracy: 0.9043 - loss: 0.2282 - val_accuracy: 0.9063 - val_loss: 0.2170
Epoch 3/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 265s 6ms/step - accuracy: 0.9048 - loss: 0.2204 - val_accuracy: 0.9027 - val_loss: 0.2198
Epoch 4/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 266s 6ms/step - accuracy: 0.9050 - loss: 0.2181 - val_accuracy: 0.9063 - val_loss: 0.2119
Epoch 5/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 266s 6ms/step - accuracy: 0.9049 - loss: 0.2157 - val_accuracy: 0.9059 - val_loss: 0.2128
Epoch 6/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 271s 6ms/step - accuracy: 0.9054 - loss: 0.2140 - val_accuracy: 0.9071 - val_loss: 0.2073
Epoch 7/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 271s 6ms/step - accuracy: 0.9059 - loss: 0.2129 - val_accuracy: 0.9366 - val_loss: 0.1735
Epoch 8/10
42509/42509 ━━━━━━━━━━━━━━━━━━━━ 272s 6ms/step - ac

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
# Metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("Accuracy: {:.10f}".format(accuracy))
print("Precision: {:.10f}".format(precision))
print("Recall: {:.10f}".format(recall))
print("F1-Score: {:.10f}".format(f1))

Accuracy: 0.9564378679
Precision: 0.9594430313
Recall: 0.9564378679
F1-Score: 0.9563173043


In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Metrics
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("Accuracy: {:.10f}".format(accuracy))
print("Precision: {:.10f}".format(precision))
print("Recall: {:.10f}".format(recall))
print("F1-Score: {:.10f}".format(f1))

10628/10628 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
Accuracy: 0.9564378679
Precision: 0.9594430313
Recall: 0.9564378679
F1-Score: 0.9563173043


**Federated Learning Simulation (FedAvg)**

In [ ]:
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Function to measure model performance
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Compute performance metrics
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)

    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = 0
    for i in range(len(local_weights)):
        # Compute the size of weights transmitted between clients and server
        communication_cost += sum(w.nbytes for w in local_weights[i])
    communication_cost += sum(w.nbytes for w in global_weights)

    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")  # in megabytes

# Federated Learning Simulation (FedAvg)
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)

    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()

    communication_overhead = []
    latency_per_round = []

    for round in range(25):  # Simulate 50 communication rounds
        local_weights = []

        start_time = time.time()  # Start measuring latency

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0)
            local_weights.append(local_model.get_weights())

        # Aggregate weights (FedAvg)
        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        # Measure latency
        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        # Measure communication overhead
        analyze_communication_overhead(local_weights, global_weights)

        # Evaluate the global model
        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")

        # Evaluate model performance metrics
        evaluate_global_model(global_model, X_test, y_test)

    # Summarize overall latency
    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 10 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Call the federated learning simulation function
federated_learning_simulation(X_train, y_train, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.7161521316
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step

Model Performance Metrics:
Accuracy: 0.7161521587
Precision: 0.7069973178
Recall: 0.7161521587
F1-Score: 0.6750516069

Confusion Matrix:
[[67066   450     3   403    91]
 [ 7643 56096   178  4083    13]
 [ 1980 47507  6034 12492     0]
 [ 4773   292  6813 56136     0]
 [ 1470  8222     0   114 58207]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.9044538140
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step

Model Performance Metrics:
Accuracy: 0.9044538413
Precision: 0.9260244435
Recall: 0.9044538413
F1-Score: 0.9012987626

Confusion Matrix:
[[67160   673     6    65   109]
 [   13 41061 26939     0     0]
 [    0     0 68013     0     0]
 [ 4308   238    93 63375     0]
 [   47     1     0     0 67965]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...


Communication Overhead: 0.77 MB

Round 23: Global Model Accuracy = 0.9062240720
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step

Model Performance Metrics:
Accuracy: 0.9062240859
Precision: 0.9292112961
Recall: 0.9062240859
F1-Score: 0.9029854637

Confusion Matrix:
[[67799   129     0     9    76]
 [   17 41057 26939     0     0]
 [    0     0 68013     0     0]
 [ 4619    38    24 63333     0]
 [   18    21     0     0 67974]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 24: Global Model Accuracy = 0.9058094621
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step

Model Performance Metrics:
Accuracy: 0.9058094605
Precision: 0.9289005519
Recall: 0.9058094605
F1-Score: 0.9025543575

Confusion Matrix:
[[67832   105     0     0    76]
 [   29 41045 26939     0     0]
 [    0     3 68010     0     0]
 [ 4793    48    13 63160     0]
 [   13     9     3     0 67988]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 25: Global Model Accuracy = 0.9069916010
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step

Model Performance Metrics:
Accuracy: 0.9069915840
Precision: 0.9299754841
Recall: 0.9069915840
F1-Score: 0.9037349157

Confusion Matrix:
[[67781   110     0    46    76]
 [   35 41037 26939     2     0]
 [    0     0 68012     1     0]
 [ 4366    20     9 63619     0]
 [   16     3     6     0 67988]]

Latency Analysis:
Average Latency per Round: 259.72 seconds
Total Latency for 10 Rounds: 6493.11 seconds


In [ ]:
!pip install cleverhans

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.5 MB/s eta 0:00:00


**Injection adversarial FGSM Attacks**

In [ ]:
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method # Import the actual function
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent # Import the actual function

# Adversarial config
USE_ADVERSARIAL = True
ATTACK_TYPE = 'fgsm'  # or 'pgd'
ADV_CLIENTS = [0, 2]  # clients to inject adversarial samples
EPSILON = 0.1

# Function to generate FGSM or PGD adversarial examples
def generate_adversarial(model, x, y, attack_type='fgsm', eps=0.1):
    if attack_type == 'fgsm':
        return fast_gradient_method(model_fn=model, x=x, eps=eps, norm=np.inf) # Call with correct arguments
    elif attack_type == 'pgd':
        return projected_gradient_descent(model_fn=model, x=x, eps=eps, eps_iter=0.01, nb_iter=40, norm=np.inf) # Call with correct arguments
    return x

# Function to evaluate the global model
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)
    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = sum(sum(w.nbytes for w in lw) for lw in local_weights)
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")

# Federated Learning Simulation with optional adversarial injection
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)
    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()
    communication_overhead = []
    latency_per_round = []

    for round in range(30):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Inject adversarial examples if enabled
            if USE_ADVERSARIAL and i in ADV_CLIENTS:
                print(f"Injecting {ATTACK_TYPE.upper()} adversarial samples into Client {i}")
                client_data[i] = generate_adversarial(local_model, client_data[i], client_labels[i], attack_type=ATTACK_TYPE, eps=EPSILON)

            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0)
            local_weights.append(local_model.get_weights())

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Call the simulation
federated_learning_simulation(X_train, y_train, X_test, y_test)

Injecting FGSM adversarial samples into Client 0
Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.5813411474
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.5813411578
Precision: 0.5888156288
Recall: 0.5813411578
F1-Score: 0.5653783842

Confusion Matrix:
[[23962 19986  5717  2905]
 [14051 28350  3472  6697]
 [ 7624 20786 17630  6530]
 [    0   268     0 52303]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.8288718462
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step

Model Performance Metrics:
Accuracy: 0.8288718429
Precision: 0.8269974513
Recall: 0.8288718429
F1-Score: 0.8251439134

Confusion Matrix:
[[38415  8640  5487    28]
 [ 8176 37424  1650  5320]
 [ 1105   413 46377  4675]
 [    0     0   491 52080]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2


...

Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 28: Global Model Accuracy = 0.8299703598
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step

Model Performance Metrics:
Accuracy: 0.8299703730
Precision: 0.8311531330
Recall: 0.8299703730
F1-Score: 0.8282944605

Confusion Matrix:
[[41251  6902  4406    11]
 [12402 36627    18  3523]
 [ 3246   266 45850  3208]
 [    0   245  1527 50799]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 29: Global Model Accuracy = 0.8504667282
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.8504667564
Precision: 0.8534695773
Recall: 0.8504667564
F1-Score: 0.8490733298

Confusion Matrix:
[[43508  5906  3130    26]
 [12408 36814     4  3344]
 [ 3177    56 47515  1822]
 [    0   202  1369 51000]]
Injecting FGSM adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting FGSM adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 30: Global Model Accuracy = 0.8141629696
6572/6572 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step

Model Performance Metrics:
Accuracy: 0.8141629534
Precision: 0.8265583717
Recall: 0.8141629534
F1-Score: 0.8136464523

Confusion Matrix:
[[43509  5892  3151    18]
 [13215 35218    41  4096]
 [ 8592    56 41081  2841]
 [    0   142  1034 51395]]

Latency Analysis:
Average Latency per Round: 262.27 seconds
Total Latency for 50 Rounds: 7868.20 seconds


**Injection adversarial PGD Attacks**

In [ ]:
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method # Import the actual function
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent # Import the actual function

# Adversarial config
USE_ADVERSARIAL = True
ATTACK_TYPE = 'pgd'  # or 'pgd'
ADV_CLIENTS = [0, 2]  # clients to inject adversarial samples
EPSILON = 0.1

# Function to generate FGSM or PGD adversarial examples
def generate_adversarial(model, x, y, attack_type='fgsm', eps=0.1):
    if attack_type == 'fgsm':
        return fast_gradient_method(model_fn=model, x=x, eps=eps, norm=np.inf) # Call with correct arguments
    elif attack_type == 'pgd':
        return projected_gradient_descent(model_fn=model, x=x, eps=eps, eps_iter=0.01, nb_iter=40, norm=np.inf) # Call with correct arguments
    return x

# Function to evaluate the global model
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)
    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = sum(sum(w.nbytes for w in lw) for lw in local_weights)
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")

# Federated Learning Simulation with optional adversarial injection
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)
    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()
    communication_overhead = []
    latency_per_round = []

    for round in range(30):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # Inject adversarial examples if enabled
            if USE_ADVERSARIAL and i in ADV_CLIENTS:
                print(f"Injecting {ATTACK_TYPE.upper()} adversarial samples into Client {i}")
                client_data[i] = generate_adversarial(local_model, client_data[i], client_labels[i], attack_type=ATTACK_TYPE, eps=EPSILON)

            local_model.fit(client_data[i], client_labels[i], epochs=1, batch_size=32, verbose=0)
            local_weights.append(local_model.get_weights())

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Call the simulation
federated_learning_simulation(X_train, y_train, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 0
Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.7909729481
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step

Model Performance Metrics:
Accuracy: 0.7909729288
Precision: 0.7919791727
Recall: 0.7909729288
F1-Score: 0.7807708364

Confusion Matrix:
[[67458   255     0   211    89]
 [13182 29443 22773  2615     0]
 [ 1119  8961 49893  8040     0]
 [ 6748  1025  4607 55634     0]
 [  103  1309     0    46 66555]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.9044185281
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step

Model Performance Metrics:
Accuracy: 0.9044185540
Precision: 0.9256115832
Recall: 0.9044185540
F1-Score: 0.9012553116

Confusion Matrix:
[[67117   735     0    71    90]
 [    3 41071 26931     8     0]
 [    0     0 67966    47     0]
 [ 4264   272    38 63440     0]
 [   19    26     0     0 67968]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 28: Global Model Accuracy = 0.9065475464
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step

Model Performance Metrics:
Accuracy: 0.9065475525
Precision: 0.9289085343
Recall: 0.9065475525
F1-Score: 0.9033342165

Confusion Matrix:
[[67610   268     0    59    76]
 [    1 41075 26937     0     0]
 [    0     0 68009     4     0]
 [ 4271    95     8 63640     0]
 [    6    55     0     0 67952]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 29: Global Model Accuracy = 0.9052595496
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step

Model Performance Metrics:
Accuracy: 0.9052595673
Precision: 0.9276913747
Recall: 0.9052595673
F1-Score: 0.9020365720

Confusion Matrix:
[[67668   258     0     1    86]
 [   11 41083 26919     0     0]
 [    0    39 67971     3     0]
 [ 4749   141     0 63124     0]
 [    0    11     0     0 68002]]
Injecting PGD adversarial samples into Client 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Injecting PGD adversarial samples into Client 2

Communication Overhead: 0.77 MB

Round 30: Global Model Accuracy = 0.9054654241
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 36s 3ms/step

Model Performance Metrics:
Accuracy: 0.9054654097
Precision: 0.9281524519
Recall: 0.9054654097
F1-Score: 0.9022369765

Confusion Matrix:
[[67690   240     0     0    83]
 [    3 41072 26938     0     0]
 [    0     0 68012     1     0]
 [ 4768    89     5 63152     0]
 [    3    18     0     0 67992]]

Latency Analysis:
Average Latency per Round: 1002.33 seconds
Total Latency for 50 Rounds: 30070.00 seconds


In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Function to measure model performance
def evaluate_global_model(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.10f}")
    print(f"Precision: {precision:.10f}")
    print(f"Recall: {recall:.10f}")
    print(f"F1-Score: {f1:.10f}")
    print("\nConfusion Matrix:")
    print(cm)

    return accuracy, precision, recall, f1

# Function to analyze communication overhead
def analyze_communication_overhead(local_weights, global_weights):
    communication_cost = 0
    for i in range(len(local_weights)):
        communication_cost += sum(w.nbytes for w in local_weights[i])
    communication_cost += sum(w.nbytes for w in global_weights)
    print(f"\nCommunication Overhead: {communication_cost / (1024 ** 2):.2f} MB")  # in MB

# FedProx Local Training Function
def train_fedprox(local_model, X, y, global_weights, mu=0.01, epochs=1, batch_size=16):
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    train_dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

    for epoch in range(epochs):
        for step, (batch_x, batch_y) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                predictions = local_model(batch_x, training=True)
                ce_loss = loss_fn(batch_y, predictions)
                prox_term = tf.add_n([tf.reduce_sum(tf.square(w - gw))
                                      for w, gw in zip(local_model.trainable_weights, global_weights)])
                total_loss = ce_loss + (mu / 2.0) * prox_term
            grads = tape.gradient(total_loss, local_model.trainable_weights)
            optimizer.apply_gradients(zip(grads, local_model.trainable_weights))
    return local_model.get_weights()

# Federated Learning Simulation (FedProx)
def federated_learning_simulation(X_train, y_train, X_test, y_test, num_clients=5):
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)

    global_model = build_hybrid_model(input_shape, num_classes)
    global_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    global_weights = global_model.get_weights()

    latency_per_round = []

    for round in range(15):
        local_weights = []
        start_time = time.time()

        for i in range(num_clients):
            local_model = build_hybrid_model(input_shape, num_classes)
            local_model.set_weights(global_weights)
            local_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            updated_weights = train_fedprox(local_model, client_data[i], client_labels[i], global_weights, mu=0.01)
            local_weights.append(updated_weights)

        global_weights = [
            np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
            for i in range(len(global_weights))
        ]
        global_model.set_weights(global_weights)

        end_time = time.time()
        latency = end_time - start_time
        latency_per_round.append(latency)

        analyze_communication_overhead(local_weights, global_weights)

        loss, acc = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"\nRound {round + 1}: Global Model Accuracy = {acc:.10f}")
        evaluate_global_model(global_model, X_test, y_test)

    print("\nLatency Analysis:")
    print(f"Average Latency per Round: {np.mean(latency_per_round):.2f} seconds")
    print(f"Total Latency for 50 Rounds: {np.sum(latency_per_round):.2f} seconds")

# Note: Make sure build_hybrid_model, input_shape, and num_classes are defined in your environment before running.
# Call the federated learning simulation function
federated_learning_simulation(X_train, y_train, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 1: Global Model Accuracy = 0.6572282910
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 39s 4ms/step

Model Performance Metrics:
Accuracy: 0.6572283027
Precision: 0.6759722460
Recall: 0.6572283027
F1-Score: 0.6251508600

Confusion Matrix:
[[54476  6146  1786  3998  1607]
 [    0  8970 24691 22987 11365]
 [    0     0 36317 31696     0]
 [ 3951   329  3411 60320     3]
 [    0     1     0  4594 63418]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 2: Global Model Accuracy = 0.9013015032
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 45s 4ms/step

Model Performance Metrics:
Accuracy: 0.9013015121
Precision: 0.9212482657
Recall: 0.9013015121
F1-Score: 0.8981534550

Confusion Matrix:
[[66278  1452    17   143   123]
 [   28 40925 27039    10    11]
 [    0     0 67925    88     0]
 [ 4257   311    52 63394     0]
 [    1    32     0     0 67980]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 6: Global Model Accuracy = 0.9022542834
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step

Model Performance Metrics:
Accuracy: 0.9022542683
Precision: 0.9224094689
Recall: 0.9022542683
F1-Score: 0.8991054146

Confusion Matrix:
[[66450  1356     4    85   118]
 [    7 40980 27010    16     0]
 [    0     0 67903   110     0]
 [ 4273   225    10 63506     0]
 [    2    24     0     0 67987]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Communication Overhead: 0.77 MB

Round 7: Global Model Accuracy = 0.9032629132
10628/10628 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step

Model Performance Metrics:
Accuracy: 0.9032628960
Precision: 0.9240957136
Recall: 0.9032628960
F1-Score: 0.9000740025

Confusion Matrix:
[[66835  1003     4    52   119]
 [   63 41004 26938     8     0]
 [    0     0 67970    43     0]
 [ 4383   233    29 63369     0]
 [    7    15     0     0 67991]]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
